In [25]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd

In [26]:
columns=['Movie_ID','Title','Director','Cast','Year','Genre']
df = pd.DataFrame(columns=columns)
df

,Movie_ID,Title,Director,Cast,Year,Genre


In [27]:
#regex module
import re
def delnum(s): # deletes fragments like [1][2][23] from string
    pattern = r'\[[^\]]*\]'
    # Match all digits in the string and replace them with an empty string
    new_string = re.sub(pattern, '', s)
    return new_string

In [28]:
URL = {
       2017:'https://en.wikipedia.org/wiki/List_of_Bangladeshi_films_of_2017'
      }
id = 2719
for key, url in URL.items():
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tbl = soup.findAll('table', {'class':"wikitable sortable","style":"width:100%;"})
    tbl += soup.findAll('table', {'class':"wikitable","style":"width:70%;"})

    for t in tbl:
        table=t.tbody
        rows = table.find_all('tr')
        for i in range(1, len(rows)):
            tds = rows[i].find_all('td')

            if len(tds)==8:  
                values = [id, tds[2].text.replace('\n',''), delnum(tds[3].text.replace('\n','')), tds[4].text.replace('\n',''), key, tds[5].text.replace('\n','')]

            elif len(tds)==7:  
                 values = [id, tds[2].text.replace('\n',''), delnum(tds[4].text.replace('\n','')), tds[5].text.replace('\n',''), key, tds[3].text.replace('\n','')]

            elif len(tds)==5:
                values = [id, tds[0].text.replace('\n',''),tds[2].text.replace('\n',''),tds[3].text.replace('\n',''), key, tds[1].text.replace('\n','')]
            
            elif len(tds)==6:
                if tds[0].text.replace('\n',"") =="JAN":
                    values = [id, tds[2].text.replace('\n',''),tds[4].text.replace('\n',''),tds[5].text.replace('\n',''), key, tds[3].text.replace('\n','')]
                else:
                    values = [id, tds[1].text.replace('\n',''),tds[3].text.replace('\n',''),tds[4].text.replace('\n',''), key, tds[2].text.replace('\n','')]

            elif len(tds)==4:
                # for k in range(4):
                #     if k!=3:
                #         print(tds[k].text.replace('\n',''),end="*")
                #     else:
                #         print(tds[k].text.replace('\n','')) 
                values = [id, tds[1].text.replace('\n',''),tds[2].text.replace('\n',''),tds[3].text.replace('\n',''), key, tds[1].text.replace('\n','')]

            id+=1
            df = df.append(pd.Series(values, index=columns), ignore_index=True)
df.shape


(34, 6)

In [29]:
df = df.apply(lambda x : x.lower().strip() if isinstance(x, str) else x)
df.iloc[df.loc[df['Title'] == ' The God of Small Things (Film) | জড়েশ্বর[26] [25][27]'].index,1] = "The God of Small Things"
df.iloc[df.loc[df['Title'] == 'The God of Small Things'].index,5] = "Drama"
df.iloc[df.loc[df['Genre'] == 'Romance-Drama'].index,5] = "Romance, Drama"
df.iloc[df.loc[df['Genre'] == 'Romance -Drama'].index,5] = "Romance, Drama"
df.iloc[df.loc[df['Title'] == 'Bengali Beauty'].index,5] = "Romance, Historical Drama"
df.iloc[df.loc[df['Title'] == 'Swatta'].index,5] = "Romance, Drama"
df.iloc[df.loc[df['Title'] == 'Voyonkor Sundor'].index,5] = "Drama"
df.to_csv('2017 bangladeshi movie list.csv', index=False)
df.shape

(34, 6)